In [4]:
import requests
import dotenv
import os
dotenv.load_dotenv()  # .env 환경변수 파일을 로드함

key = os.getenv("SEOUL_SUBWAY_API")

In [5]:
# 지하철 실시간 도착정보  https://data.seoul.go.kr/dataList/OA-12764/F/1/datasetView.do
# http://swopenAPI.seoul.go.kr/api/subway/(인증키)/xml/realtimeStationArrival/0/5/서울

url = f'http://swopenAPI.seoul.go.kr/api/subway/{key}/json/realtimeStationArrival/0/100/강남'
response = requests.get(url)
response.status_code

200

In [6]:
respnse = response.json()
for row in respnse['realtimeArrivalList']:
    print(f'열차종류:{row["btrainSttus"]} 도착예정시간:{int(row["barvlDt"])//60}분 {int(row["barvlDt"])%60}초')
    print(row['trainLineNm'],row['statnNm'],row['statnFid'],row['statnId'],row['statnTid'])
    print('*'*100)   

열차종류:일반 도착예정시간:0분 0초
신사행 - 신논현방면 강남 1077000688 1077000687 1077000686
****************************************************************************************************
열차종류:일반 도착예정시간:2분 40초
성수행 - 역삼방면 강남 1002000223 1002000222 1002000221
****************************************************************************************************
열차종류:일반 도착예정시간:7분 0초
성수행 - 역삼방면 강남 1002000223 1002000222 1002000221
****************************************************************************************************
열차종류:일반 도착예정시간:0분 0초
신사행 - 신논현방면 강남 1077000688 1077000687 1077000686
****************************************************************************************************
열차종류:일반 도착예정시간:0분 0초
광교행 - 양재방면 강남 1077000686 1077000687 1077000688
****************************************************************************************************
열차종류:일반 도착예정시간:4분 0초
성수행 - 교대방면 강남 1002000221 1002000222 1002000223
************************************************************************************

In [7]:
line2_stations = [
    # 본선
    "시청", "을지로입구", "을지로3가", "을지로4가", "동대문역사문화공원", "신당", "상왕십리",
    "왕십리", "한양대", "뚝섬", "성수", "건대입구", "구의", "강변", "잠실나루", "잠실",
    "잠실새내", "종합운동장", "삼성", "선릉", "역삼", "강남", "교대", "서초", "방배", 
    "사당", "낙성대", "서울대입구", "봉천", "신림", "신대방", "구로디지털단지", "대림",
    "신도림", "문래", "영등포구청", "당산", "합정", "홍대입구", "신촌", "이대", "아현",
    "충정로", "시청",
    # 성수지선
    "성수", "용답", "신답", "용두", "신설동",
    # 신정지선
    "신도림", "도림천", "양천구청", "신정네거리", "까치산"
]

In [8]:
# 역명 - 역명코드  매핑테이블 만들기
mappings = {}
for station in line2_stations:
    url = f'http://swopenAPI.seoul.go.kr/api/subway/{key}/json/realtimeStationArrival/0/100/{station}'
    response = requests.get(url)    
    respnse = response.json()
    values = []
    for row in respnse['realtimeArrivalList']:                        
        values.append( row['statnId'] )
    mappings[station] = set(values)
mappings 


{'시청': {'1001000132', '1002000201'},
 '을지로입구': {'1002000202'},
 '을지로3가': {'1002000203', '1003000330'},
 '을지로4가': {'1002000204', '1005000535'},
 '동대문역사문화공원': {'1002000205', '1004000422', '1005000536'},
 '신당': {'1002000206', '1006000635'},
 '상왕십리': {'1002000207'},
 '왕십리': {'1002000208', '1005000540', '1063075116', '1075075210'},
 '한양대': {'1002000209'},
 '뚝섬': {'1002000210'},
 '성수': {'1002000211'},
 '건대입구': {'1002000212', '1007000727'},
 '구의': {'1002000213'},
 '강변': {'1002000214'},
 '잠실나루': {'1002000215'},
 '잠실': {'1002000216', '1008000814'},
 '잠실새내': {'1002000217'},
 '종합운동장': {'1002000218', '1009000930'},
 '삼성': {'1002000219'},
 '선릉': {'1002000220', '1075075215'},
 '역삼': {'1002000221'},
 '강남': {'1002000222', '1077000687'},
 '교대': {'1002000223', '1003000340'},
 '서초': {'1002000224'},
 '방배': {'1002000225'},
 '사당': {'1002000226', '1004000433'},
 '낙성대': {'1002000227'},
 '서울대입구': {'1002000228'},
 '봉천': {'1002000229'},
 '신림': {'1002000230', '1094000408'},
 '신대방': {'1002000231'},
 '구로디지털단지': {'1